In [ ]:
import tensorflow as tf
tf.compat.v1.ConfigProto()
config = tf.compat.v1.ConfigProto(gpu_options=tf.compat.v1.GPUOptions(allow_growth=True))

gpu_options=tf.compat.v1.GPUOptions(allow_growth=True)
sess = tf.compat.v1.Session(config=config)

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import keras
import random
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, Flatten
from tensorflow.compat.v1.keras.layers import CuDNNGRU, CuDNNLSTM
from keras.layers import AveragePooling1D, MaxPooling1D, Bidirectional, GlobalMaxPool1D, Concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D,concatenate
from keras.layers import SpatialDropout1D
from keras.models import Sequential
from keras.callbacks import Callback
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import Callback
import matplotlib.pyplot as plt
import csv
import os
import time
from math import floor
import warnings
from keras import backend as K
warnings.filterwarnings('ignore')
np.random.seed(42)
random.seed(12345)

In [ ]:
if tf.config.list_physical_devices('GPU'):
    print("GPU is available.")
    print("TensorFlow version:", tf.__version__)
    gpus = tf.config.list_physical_devices('GPU')
    print("Number of available GPUs:", len(gpus))
    for i in range(len(gpus)):
        print("GPU name:", gpus[i].name)
else:
    print("GPU is not available.")

In [ ]:
with open('....pkl', 'rb') as f:
    Data = pickle.load(f)

In [ ]:
iput_class = 2000
oput_class = 2000

In [ ]:
aug_time=200
numtrain = iput_class * aug_time
numval = int(0.2 * iput_class)
# Extract features and labels
X = Data.iloc[:, 1:].values
y = Data.iloc[:, 0].values

# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split your data into training and validation sets:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
def log_to_csv(log_file,early_stopping_setting, epoch, elapsed_time, top1, top3, top5):
    # Check if file exists
    if not os.path.isfile(log_file):
        # Open file in write mode and write headers
        with open(log_file, mode='w') as f:
            writer = csv.writer(f)
            writer.writerow(["early_stopping_setting","Epoch", "Elapsed_Time", "Top1", "Top3", "Top5"])
            
    # File exists, open in append mode and write data
    with open(log_file, mode='a') as f:
        writer = csv.writer(f)
        writer.writerow([early_stopping_setting,epoch, elapsed_time, top1, top3, top5])
class CustomEarlyStopping(Callback):
    def __init__(self, patience=0, early_step_setting=0.95):
        super(CustomEarlyStopping, self).__init__()
        self.patience = patience
        self.early_step_setting = early_step_setting
        self.wait = 0
        self.stopped_epoch = 0
        self.best = 0
        self.losses = []

    def on_train_begin(self, logs=None):
        self.wait = 0
        self.stopped_epoch = 0
        self.best = np.Inf

    def on_epoch_end(self, epoch, logs=None):
        current_loss = logs.get('loss')
        self.losses.append(current_loss)

        if epoch > 5 and current_loss > self.losses[epoch - 1] * self.early_step_setting:
            self.wait += 1
            print('\n',self.wait)
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print("Early stopping!")
def calculate_top_k_accuracy(y_true, y_pred, k):
    top_k_preds = np.argsort(y_pred, axis=-1)[:, -k:]
    target_in_top_k = [(true in top_k) for true, top_k in zip(y_true, top_k_preds)]
    return np.mean(target_in_top_k)                

In [ ]:
from datetime import datetime
for iiiii in range(100):
    keras.backend.clear_session()
    # Construct CNN layers
    # more para-LeNet-5
    cnn_model = Sequential()
    cnn_model.add(Conv1D(6, 5, strides = 1, activation = 'relu', input_shape = (2251, 1)))
    cnn_model.add(MaxPooling1D(2, strides = 2))
    cnn_model.add(Dropout(0.2))

    cnn_model.add(Conv1D(16, 5, strides = 1, activation = 'relu'))
    cnn_model.add(Conv1D(16, 5, strides = 1, activation = 'relu'))
    cnn_model.add(MaxPooling1D(2, strides = 2))
    cnn_model.add(Dropout(0.2))

    cnn_model.add(Conv1D(32, 5, strides = 1, activation = 'relu'))
    cnn_model.add(Conv1D(32, 5, strides = 1, activation = 'relu'))
    cnn_model.add(MaxPooling1D(2, strides = 2))
    cnn_model.add(Dropout(0.2))

    cnn_model.add(Conv1D(64, 5, strides = 1, activation = 'relu'))
    cnn_model.add(Conv1D(64, 5, strides = 1, activation = 'relu'))
    cnn_model.add(MaxPooling1D(2, strides = 2))
    cnn_model.add(Dropout(0.2))

    cnn_model.add(Flatten())
    cnn_model.add(Dense(120, activation = 'relu'))
    cnn_model.add(Dense(84, activation = 'relu'))
    cnn_model.add(Dense(oput_class, activation = 'softmax', activity_regularizer = keras.regularizers.l2(0.1)))
    cnn_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

    #cnn_model.summary()

    # Convert labels to one-hot encoding
    y_train_onehot = to_categorical(y_train)
    y_val_onehot = to_categorical(y_val)

    # Set up early stopping
    #early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)


    early_stopping_setting=1
    custom_early_stopping = CustomEarlyStopping(patience=3, early_step_setting=early_stopping_setting)
    start_time = time.time()
    cnn_model.fit(
        #X_train.numpy().reshape(X_train.shape[0], X_train.shape[1], 1), 
        X_train.reshape(X_train.shape[0], X_train.shape[1], 1), 

        y_train_onehot, 
        batch_size = 700, 
        epochs = 30, 
        verbose = 1,
        validation_data=(X_val.reshape(X_val.shape[0], X_val.shape[1], 1), y_val_onehot),
        callbacks=[custom_early_stopping]
    )
    elapsed_time = time.time() - start_time



    # Make predictions
    y_pred = cnn_model.predict(X_val.reshape(X_val.shape[0], X_val.shape[1], 1))

    # Calculate top-1, top-3, top-5 accuracies
    top_1_accuracy = calculate_top_k_accuracy(y_val, y_pred, 1)
    top_3_accuracy = calculate_top_k_accuracy(y_val, y_pred, 3)
    top_5_accuracy = calculate_top_k_accuracy(y_val, y_pred, 5)

    print(f"Top-1 Accuracy: {round(top_1_accuracy, 2) * 100}%")
    print(f"Top-3 Accuracy: {round(top_3_accuracy, 2) * 100}%")
    print(f"Top-5 Accuracy: {round(top_5_accuracy, 2) * 100}%")

    # Plot training loss
    #plt.plot(custom_early_stopping.losses)
    #plt.title('Model loss')
    #plt.ylabel('Loss')
    #plt.xlabel('Epoch')
    #plt.legend(['Train'], loc='upper right')
    #plt.show()
    #from sklearn.preprocessing import StandardScaler

    # Fit the scaler on the training data
    #scaler = StandardScaler()

    Test = pd.read_csv('....csv')

    X_test = Test.iloc[:,1:]
    y_true = Test.iloc[:,0]

    per_sample_results = []
    
    for i in range(len(X_test)):
        x = X_test.iloc[i].values.reshape(1, 2251, 1)
        y_true_sample = y_true[i]

        y_pred_sample = cnn_model.predict(x)
        
        # Get the top k predictions
        top_1_preds = np.argsort(y_pred_sample, axis=-1)[:, -1:]
        top_3_preds = np.argsort(y_pred_sample, axis=-1)[:, -3:]
        top_5_preds = np.argsort(y_pred_sample, axis=-1)[:, -5:]

        true_label_encoded = label_encoder.transform([y_true_sample])

        # Initialize counters
        correct_top1 = correct_top3 = correct_top5 = 0

        if true_label_encoded in top_1_preds[0]:
            correct_top1 = 1

        if true_label_encoded in top_3_preds[0]:
            correct_top3 = 1

        if true_label_encoded in top_5_preds[0]:
            correct_top5 = 1

        per_sample_results.extend([correct_top1, correct_top3, correct_top5])
    
    current_time = datetime.now().strftime("%Y%m%d-%H%M%S")
    '''...'''